# 附录 10.2: 工具使用

- [课程](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
# 安装 anthropic 库
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 变量
%store -r API_KEY

# 初始化 Anthropic 客户端
client = anthropic.Anthropic(api_key=API_KEY)

# 重写以调用 Claude 3 Sonnet，它在工具使用方面通常表现更好，并包含停止序列
def get_completion(messages, system_prompt="", prefill="", stop_sequences=None):
    """
    获取 Claude 模型的完成响应
    
    参数:
    - messages: 消息列表
    - system_prompt: 系统提示词
    - prefill: 预填充内容
    - stop_sequences: 停止序列列表
    
    返回:
    - 模型生成的文本响应
    """
    message = client.messages.create(
        model="claude-3-sonnet-20240229",  # 使用 Claude 3 Sonnet 模型
        max_tokens=2000,                   # 最大令牌数
        temperature=0.0,                   # 温度设置为0，确保确定性输出
        system=system_prompt,              # 系统提示词
        messages=messages,                 # 对话消息列表
        stop_sequences=stop_sequences      # 停止序列
    )
    return message.content[0].text

---

## 课程

虽然乍一看可能在概念上比较复杂，但工具使用（也叫函数调用）实际上非常简单！您已经掌握了实现工具使用所需的所有技能，它实际上只是替换和提示链的组合。

在之前的替换练习中，我们将文本替换到提示中。而在工具使用中，我们将工具或函数的结果替换到提示中。Claude 不能直接调用或访问工具和函数。相反，我们让 Claude：
1. 输出它想要调用的工具名称和参数
2. 在调用工具时暂停任何进一步的响应生成
3. 然后我们用附加的工具结果重新提示

函数调用很有用，因为它扩展了 Claude 的能力，使 Claude 能够处理更复杂的多步骤任务。
您可以给 Claude 的一些函数示例：
- 计算器
- 单词计数器
- SQL 数据库查询和数据检索
- 天气 API

您可以通过结合以下两个元素让 Claude 进行工具使用：

1. 系统提示词，在其中我们向 Claude 解释工具使用的概念，以及它可以访问的工具的详细描述列表
2. 控制逻辑，用于编排和执行 Claude 的工具使用请求

### 工具使用路线图

*本课程教授我们当前的工具使用格式。但是，我们将在不久的将来更新和改进工具使用功能，包括：*
* *更简化的函数定义和调用格式*
* *更强健的错误处理和边缘情况覆盖*
* *与我们 API 其余部分的更紧密集成*
* *更好的可靠性和性能，特别是对于更复杂的工具使用任务*

### 示例

要在 Claude 中启用工具使用，我们从系统提示词开始。在这个特殊的工具使用系统提示词中，我们告诉 Claude：
* 工具使用的基本前提和它所涉及的内容
* Claude 如何调用和使用给定的工具
* 在这个特定场景中它可以访问的工具的详细列表

这是系统提示词的第一部分，向 Claude 解释工具使用。系统提示词的这一部分在所有 Claude 工具使用的实例中都是通用的。我们给 Claude 的工具调用结构（`<function_calls> [...] </function_calls>`）是 Claude 专门训练使用的结构，所以我们建议您坚持使用这个结构。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

这是系统提示词的第二部分，它定义了 Claude 在这个特定情况下可以访问的确切工具。在这个示例中，我们将给 Claude 一个计算器工具，它需要三个参数：两个操作数和一个运算符。

然后我们将系统提示词的两个部分组合起来。

In [ ]:
# 系统提示词的特定工具部分 - 定义计算器工具
system_prompt_tools_specific_tools = """以下是 JSONSchema 格式的可用函数：
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
用于进行基本算术运算的计算器函数。
支持加法、减法、乘法和除法
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第一个操作数（运算符之前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第二个操作数（运算符之后）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>要执行的运算。必须是 +、-、* 或 / 中的一个</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

# 组合通用说明和特定工具定义
system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

现在我们可以给 Claude 一个需要使用 `calculator` 工具的问题。我们将在 `stop_sequences` 中使用 `<function_calls\>` 来检测 Claude 是否以及何时调用函数。

In [ ]:
# 创建一个需要乘法运算的用户消息
multiplication_message = {
    "role": "user",
    "content": "计算 1,984,135 乘以 9,343,116"
}

# 设置停止序列来检测函数调用的结束
stop_sequences = ["</function_calls>"]

# 获取 Claude 的响应
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

现在，我们可以从 Claude 的函数调用中提取参数，并代表 Claude 实际运行函数。

首先我们定义函数的代码。

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    """
    执行两个数字之间的基本算术运算
    
    参数:
    - num1: 第一个操作数
    - num2: 第二个操作数  
    - operation: 运算符 (+, -, *, /)
    
    返回:
    - 运算结果或错误信息
    """
    if operation == '+':
        return num1 + num2      # 加法
    elif operation == "-":
        return num1 - num2      # 减法
    elif operation == "*":
        return num1 * num2      # 乘法
    elif operation == "/":
        return num1 / num2      # 除法
    else:
        return "错误：不支持的运算。"

然后我们从 Claude 的函数调用响应中提取参数。如果所有参数都存在，我们就运行计算器工具。

In [ ]:
def find_parameter(message, parameter_name):
    """
    从 Claude 的响应消息中提取指定的参数值
    
    参数:
    - message: Claude 的响应消息
    - parameter_name: 要查找的参数名称
    
    返回:
    - 参数值或 None（如果未找到）
    """
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

# 从 Claude 的响应中提取函数参数
first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

# 如果所有参数都存在，执行计算
if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- 结果 ----------------")
    print(f"{result:,}")

现在我们有了结果，我们必须正确格式化该结果，以便当我们将其传回给 Claude 时，Claude 能够理解该结果与哪个工具相关。Claude 已经被训练识别以下固定格式：
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

运行下面的单元格将上述工具结果格式化为这种结构。

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    """
    构造成功的函数运行结果注入提示
    
    参数:
    - invoke_results: 包含工具名称和结果的字典列表
    
    返回:
    - 格式化的函数结果字符串
    """
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

# 格式化工具结果
formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',  # 工具名称
    'tool_result': result                   # 工具执行结果
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

现在我们要做的就是通过将结果附加到之前相同的消息链中，将这个结果发送回 Claude，就完成了！

In [ ]:
# 完整的第一次响应（添加结束标签）
full_first_response = function_calling_response + "</function_calls>"

# 构造完整的对话
messages = [multiplication_message,
{
    "role": "assistant",           # Claude 的响应
    "content": full_first_response
},
{
    "role": "user",               # 用户提供的函数结果
    "content": function_results
}]
   
# 打印 Claude 的最终响应
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- 最终结果 -------------")
print(final_response)

恭喜您端到端地运行了完整的工具使用链！

那么如果我们给 Claude 一个根本不需要使用给定工具的问题会怎样呢？

In [ ]:
# 创建一个不需要计算的问题
non_multiplication_message = {
    "role": "user",
    "content": "告诉我法国的首都是什么。"
}

stop_sequences = ["</function_calls>"]

# 获取 Claude 的响应
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

成功！如您所见，Claude 知道在不需要时不调用函数。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问 [**示例练习场**](#example-playground)。

---

## 练习
- [练习 10.2.1 - SQL](#exercise-1021---SQL)

### 练习 10.2.1 - SQL
在这个练习中，您将编写一个工具使用提示，用于查询和写入世界上最小的"数据库"。这是初始化的数据库，实际上只是一个字典。

In [ ]:
# 初始化一个简单的"数据库"（实际上是一个字典）
db = {
    "users": [                                                    # 用户表
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [                                                 # 产品表
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

以下是读取和写入数据库的函数代码。

In [ ]:
def get_user(user_id):
    """
    根据用户ID获取用户信息
    
    参数:
    - user_id: 用户ID
    
    返回:
    - 用户字典或None（如果未找到）
    """
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    """
    根据产品ID获取产品信息
    
    参数:
    - product_id: 产品ID
    
    返回:
    - 产品字典或None（如果未找到）
    """
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    """
    向数据库添加新用户
    
    参数:
    - name: 用户姓名
    - email: 用户邮箱
    
    返回:
    - 新创建的用户字典
    """
    user_id = len(db["users"]) + 1              # 生成新的用户ID
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)                    # 添加到用户列表
    return user

def add_product(name, price):
    """
    向数据库添加新产品
    
    参数:
    - name: 产品名称
    - price: 产品价格
    
    返回:
    - 新创建的产品字典
    """
    product_id = len(db["products"]) + 1        # 生成新的产品ID
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)              # 添加到产品列表
    return product

要解决这个练习，首先定义一个类似于上面 `system_prompt_tools_specific_tools` 的系统提示。确保包含每个工具的名称和描述，以及每个函数的每个参数的名称、类型和描述。我们在下面给您提供了一些起始框架。

In [ ]:
# 练习：为 SQL 数据库操作定义工具提示
system_prompt_tools_specific_tools_sql = """
# 在这里定义您的SQL工具...
# 提示：需要定义 get_user, get_product, add_user, add_product 四个工具
"""

# 组合通用说明和SQL特定工具定义
system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

准备好后，您可以在下面的示例上测试您的工具定义系统提示。只需运行下面的单元格！

In [ ]:
# 测试示例
examples = [
    "向数据库添加一个名为 Deborah 的用户。",
    "向数据库添加一个名为 Thingo 的产品",
    "告诉我用户2的姓名",
    "告诉我产品3的名称"
]

# 遍历每个示例进行测试
for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # 获取并打印 Claude 的响应
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

如果您做对了，函数调用消息应该正确调用 `add_user`、`add_product`、`get_user` 和 `get_product` 函数。

作为额外练习，添加一些代码单元格并编写参数解析代码。然后使用 Claude 给您的参数调用函数，以查看调用后"数据库"的状态。

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
# 运行此单元格查看练习解决方案
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### 恭喜！

恭喜您学会了工具使用和函数调用！如果您想了解更多关于搜索和 RAG 的内容，请前往最后的附录部分。

---

## 示例练习场

这是一个供您自由试验本课程中显示的提示示例的区域，您可以调整提示来看看这可能如何影响 Claude 的响应。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
# 系统提示词的特定工具部分 - 定义计算器工具（示例练习场版本）
system_prompt_tools_specific_tools = """以下是 JSONSchema 格式的可用函数：
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
用于进行基本算术运算的计算器函数。
支持加法、减法、乘法和除法
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第一个操作数（运算符之前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第二个操作数（运算符之后）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>要执行的运算。必须是 +、-、* 或 / 中的一个</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

# 组合通用说明和特定工具定义（示例练习场版本）
system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
# 创建一个需要乘法运算的用户消息（示例练习场版本）
multiplication_message = {
    "role": "user",
    "content": "计算 1,984,135 乘以 9,343,116"
}

# 设置停止序列来检测函数调用的结束
stop_sequences = ["</function_calls>"]

# 获取 Claude 的响应
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    """
    执行两个数字之间的基本算术运算（示例练习场版本）
    
    参数:
    - num1: 第一个操作数
    - num2: 第二个操作数  
    - operation: 运算符 (+, -, *, /)
    
    返回:
    - 运算结果或错误信息
    """
    if operation == '+':
        return num1 + num2      # 加法
    elif operation == "-":
        return num1 - num2      # 减法
    elif operation == "*":
        return num1 * num2      # 乘法
    elif operation == "/":
        return num1 / num2      # 除法
    else:
        return "错误：不支持的运算。"

In [ ]:
def find_parameter(message, parameter_name):
    """
    从 Claude 的响应消息中提取指定的参数值（示例练习场版本）
    
    参数:
    - message: Claude 的响应消息
    - parameter_name: 要查找的参数名称
    
    返回:
    - 参数值或 None（如果未找到）
    """
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

# 从 Claude 的响应中提取函数参数
first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

# 如果所有参数都存在，执行计算
if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- 结果 ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    """
    构造成功的函数运行结果注入提示（示例练习场版本）
    
    参数:
    - invoke_results: 包含工具名称和结果的字典列表
    
    返回:
    - 格式化的函数结果字符串
    """
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

# 格式化工具结果
formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',  # 工具名称
    'tool_result': result                   # 工具执行结果
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
# 完整的第一次响应（添加结束标签）
full_first_response = function_calling_response + "</function_calls>"

# 构造完整的对话
messages = [multiplication_message,
{
    "role": "assistant",           # Claude 的响应
    "content": full_first_response
},
{
    "role": "user",               # 用户提供的函数结果
    "content": function_results
}]
   
# 打印 Claude 的最终响应
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- 最终结果 -------------")
print(final_response)

In [ ]:
# 创建一个不需要计算的问题（示例练习场版本）
non_multiplication_message = {
    "role": "user",
    "content": "告诉我法国的首都是什么。"
}

stop_sequences = ["</function_calls>"]

# 获取 Claude 的响应
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)